# Sweep Explorer - Interactive Parameter Visualization

Interactive explorer for experimental sweep results. Map sweep parameters
(entry_max, label_max, dataset, summarizer, k) onto chart axes, facets,
color, and filters.

**Prerequisites:** Run `scripts/run_experimental_sweep.py` or `pca_kllmeans_sweep.ipynb`
to generate `experimental_sweep_*.pkl` files in the repo root.

In [ ]:
# Install hvplot if needed (safe to re-run)
%pip install -q hvplot

import pickle
import warnings
from pathlib import Path

import numpy as np
import pandas as pd

import hvplot.pandas  # registers .hvplot accessor on DataFrames
import panel as pn

# Enable Panel widgets in notebook
pn.extension('tabulator', design='material', sizing_mode='stretch_width')

warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
def load_sweep_dataframe(data_dir: str = "../experimental_results") -> pd.DataFrame:
    """
    Load all experimental_sweep_*.pkl files and flatten into one row per (file, k).

    Args:
        data_dir: Directory containing pickle files (default: ../experimental_results)

    Returns:
        DataFrame with sweep parameters and metrics as columns.
    """
    rows = []
    errors = []

    for pkl_path in sorted(Path(data_dir).glob("experimental_sweep_*.pkl")):
        try:
            with open(pkl_path, "rb") as f:
                data = pickle.load(f)
        except Exception as e:
            errors.append((pkl_path.name, str(e)))
            continue

        meta = data.get("metadata", {})
        result = data.get("result", {})
        by_k = result.get("by_k", {})

        for k_str, k_data in by_k.items():
            stab = k_data.get("stability") or {}
            # Handle StabilityMetrics dataclass instances vs plain dicts
            if hasattr(stab, "__dict__") and not isinstance(stab, dict):
                stab = stab.__dict__

            rows.append({
                "entry_max": meta.get("entry_max"),
                "dataset": meta.get("benchmark_source", data.get("dataset_name", "unknown")),
                "label_max": meta.get("label_max"),
                "summarizer": meta.get("summarizer", "unknown"),
                "k": int(k_str),
                "ari_mean": stab.get("stability_ari_mean"),
                "ari_std": stab.get("stability_ari_std"),
                "silhouette_mean": stab.get("silhouette_mean"),
                "silhouette_std": stab.get("silhouette_std"),
                "coverage_mean": stab.get("coverage_mean"),
                "coverage_std": stab.get("coverage_std"),
                "inertia_mean": stab.get("inertia_mean"),
                "inertia_std": stab.get("inertia_std"),
                "ari_vs_ground_truth": stab.get("ari_vs_ground_truth"),
                "objective": k_data.get("objective"),
                "actual_entry_count": meta.get("actual_entry_count"),
                "actual_label_count": meta.get("actual_label_count"),
                "source_file": pkl_path.name,
            })

    if errors:
        print(f"[WARN] Failed to load {len(errors)} file(s):")
        for name, err in errors[:5]:
            print(f"  {name}: {err}")

    df = pd.DataFrame(rows)
    if not df.empty:
        print(f"[OK] Loaded {len(df)} rows from {df['source_file'].nunique()} pickle files.")
        print(f"     Datasets: {sorted(df['dataset'].unique())}")
        print(f"     Summarizers: {sorted(df['summarizer'].unique())}")
        print(f"     entry_max: {sorted(df['entry_max'].unique())}")
        print(f"     label_max: {sorted(df['label_max'].unique())}")
        print(f"     k range: {df['k'].min()} - {df['k'].max()}")
    else:
        print("[WARN] No pickle files found. Run sweep script first to generate data.")
    return df


df = load_sweep_dataframe()
df.head()

## Interactive Explorer

The explorer below lets you change:
- **Kind**: Chart type (line, scatter, bar, box, etc.)
- **X / Y**: Which column on each axis
- **by**: Color grouping
- **groupby**: Creates a widget (slider/dropdown) to step through values
- Additional options in the sidebar (aggregation, faceting, etc.)

If `hvDataFrameExplorer` is not available in your hvplot version, Cell 5 provides
a manual widget-based fallback.

In [ ]:
# Try the built-in explorer first
try:
    from hvplot.ui import hvDataFrameExplorer
except ImportError:
    try:
        from hvplot.explorer import hvDataFrameExplorer
    except ImportError:
        hvDataFrameExplorer = None

if hvDataFrameExplorer is not None and not df.empty:
    explorer = hvDataFrameExplorer(
        df,
        kind="line",
        x="k",
        y="ari_mean",
        by=["summarizer"],
    )
    explorer  # display in notebook
else:
    if df.empty:
        print("[WARN] No data loaded. Run sweep first.")
    else:
        print("[INFO] hvDataFrameExplorer not available. Use the manual explorer in the next cell.")

In [ ]:
# Manual explorer: Panel widgets + hvplot (works in any hvplot version)
# Run this cell if hvDataFrameExplorer is not available, or if you want more control.

if df.empty:
    print("[WARN] No data loaded.")
else:
    numeric_cols = list(df.select_dtypes(include="number").columns)
    categorical_cols = [c for c in df.columns if c not in numeric_cols]
    all_cols = numeric_cols + categorical_cols

    # --- Chart controls ---
    kind_select = pn.widgets.Select(name="Chart Type", options=["line", "scatter", "bar", "box", "area", "hist"], value="line", width=140)
    x_select = pn.widgets.Select(name="X Axis", options=numeric_cols, value="k", width=180)
    y_select = pn.widgets.Select(name="Y Axis", options=numeric_cols, value="ari_mean", width=180)
    by_select = pn.widgets.Select(name="Color (by)", options=["None"] + all_cols, value="summarizer", width=180)
    facet_col = pn.widgets.Select(name="Facet Column", options=["None"] + categorical_cols, value="None", width=180)
    facet_row = pn.widgets.Select(name="Facet Row", options=["None"] + categorical_cols, value="None", width=180)

    # --- Filter widgets for sweep parameters ---
    filter_widgets = {}
    for col in ["entry_max", "label_max", "dataset", "summarizer"]:
        if col in df.columns:
            unique_vals = sorted(df[col].dropna().unique(), key=str)
            if len(unique_vals) <= 30:
                filter_widgets[col] = pn.widgets.MultiChoice(
                    name=f"Filter: {col}",
                    options=[str(v) for v in unique_vals],
                    value=[str(v) for v in unique_vals],
                    width=300,
                )

    plot_pane = pn.pane.HoloViews(None, sizing_mode="stretch_width", min_height=450)

    def update_plot(*events):
        filtered = df.copy()

        # Apply filters
        for col, widget in filter_widgets.items():
            if widget.value:
                filtered = filtered[filtered[col].astype(str).isin(widget.value)]

        if filtered.empty:
            plot_pane.object = None
            return

        kwargs = {
            "kind": kind_select.value,
            "x": x_select.value,
            "y": y_select.value,
            "responsive": True,
            "height": 400,
        }
        if by_select.value != "None":
            kwargs["by"] = by_select.value
        if facet_col.value != "None":
            kwargs["col"] = facet_col.value
        if facet_row.value != "None":
            kwargs["row"] = facet_row.value

        try:
            plot = filtered.hvplot(**kwargs)
            plot_pane.object = plot
        except Exception as e:
            print(f"Plot error: {e}")
            plot_pane.object = None

    # Wire widgets
    for w in [kind_select, x_select, y_select, by_select, facet_col, facet_row]:
        w.param.watch(update_plot, "value")
    for w in filter_widgets.values():
        w.param.watch(update_plot, "value")

    # Initial plot
    update_plot()

    # Layout
    controls = pn.Column(
        pn.pane.Markdown("### Chart Controls"),
        pn.Row(kind_select, x_select, y_select, by_select),
        pn.Row(facet_col, facet_row),
        pn.pane.Markdown("### Filters"),
        *list(filter_widgets.values()),
    )

    pn.Column(controls, plot_pane)

## Quick Preset Plots

The cells below provide one-liner hvplot calls for common views.
Modify them as needed.

In [ ]:
if not df.empty:
    df.hvplot.line(
        x="k", y="ari_mean", by="summarizer",
        groupby=["dataset", "entry_max", "label_max"],
        title="ARI vs K",
        height=400, responsive=True,
    )

In [ ]:
if not df.empty:
    df.hvplot.heatmap(
        x="k", y="label_max", C="silhouette_mean",
        groupby=["dataset", "summarizer", "entry_max"],
        cmap="viridis", title="Silhouette Mean",
        height=400, responsive=True,
    )

In [ ]:
if not df.empty:
    df.hvplot.box(
        y="ari_mean", by="dataset",
        groupby=["entry_max", "label_max"],
        title="ARI Distribution by Dataset",
        height=400, responsive=True,
    )